In [1]:
import pandas as pd
import numpy as np
import re
#全角を半角、半角を全角にできるライブラリ 漢字はそのままでアルファベット、カタカナだけを変換してくれる。
import mojimoji

In [343]:
df = pd.read_csv('../data/df_info_20231115.csv',index_col=0).reset_index().drop(columns='index')

In [344]:
#保証期間のヶ月を取り除く
#df['保証期間'] = df['保証期間'].str.replace('ヶ月','')

#データに年とヶ月が混ざっているため、ヶ月を年に変換()
df['保証期間'] = df['保証期間'].astype(str)
def grantee_period_replace(text):
    if text[1:] == 'ヶ月':
        text = int(text[0])/12
    return text
df['保証期間']=df['保証期間'].apply(grantee_period_replace)
#保証期間が'-'のものは0を入れておく
df['保証期間']=df['保証期間'].replace('-',0)
df['保証期間'] = df['保証期間'].astype(float)

In [345]:
#初年度登録年月、次回車検、入荷日は半数が欠損値となってしまっているためと削除
df.drop(columns=['初年度登録年月','次回車検','入荷日','備考'],inplace=True)

In [346]:
df

,id,価格,車両名,メーカー,型式番号,モデル年,排気量,色,タイプ,走行距離,保証期間,在庫店舗,店舗在庫
0,2100006155617,118000,HONDA トゥデイ,HONDA,JBH-AF67,2012,50,黒,スクーター/ビッグスクーター,2742,3,京都店,0120-37-8190
1,2100004366077,1378000,HONDA 【マル得】CB400SFV-4ABS,HONDA,EBL-NC42,2019,400,黒,ネイキッド,776,5,仙台店,0120-37-8190
2,2100005457217,348000,KAWASAKI Z250,KAWASAKI,JBK-ER250C,2013,250,白,ネイキッド,11979,1,りんくうシークル店,0120-37-8190
3,2100005410007,578000,YAMAHA YZF-R3 Monster Energy Yamaha MotoGP Edi...,YAMAHA,EBL-RH07J,2020,320,黒,スポーツ/ツアラー,4500,5,金沢店,0120-37-8190
4,2100005346689,568000,Husqvarna SVARTPILEN 250【マル得車両】 2021年モデル／バーエンドミラー,Husqvarna,JUE4,2021,250,黒,ネイキッド,5688,5,Husqvarna Motorcycles いわき店,0120-37-8190
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6102,2100006097726,1018000,YAMAHA YZF-R7,YAMAHA,8BL-RM39J,2022,700,青/黒,スポーツ/ツアラー,2964,7,広島店,0120-37-8190
6103,2100006214697,798000,SUZUKI DR-Z400SM★最終型★,SUZUKI,BC-SK44A,2008,400,黒,オフロード/モタード,11745,3,ラパークいわき店,0120-37-8190
6104,2100006202625,1198000,HONDA CBR1000RR ABS Special Edition,HONDA,EBL-SC59,2013,1000,バイブラントオレンジ,スポーツ/ツアラー,23013,1,久留米店,0120-37-8190
6105,2100006271508,466000,YAMAHA YZF-R25【マル得】2019年式モデル/マルチバー/USB電源！!,YAMAHA,JBK-RG10J,2019,250,艶消し赤,スポーツ/ツアラー,5895,5,東大阪店,0120-37-8190


# 名称、型式番号加工

バイクの名前(車種)がうまく取得できていない部分がある(備考に入れたかった内容やメーカー名が入ってきてしまっている)  
メーカー名をfor文とreplaceを使って削除。  
色をある程度統一したい(艶消し赤やグランプリレッドなどは一律で赤にするなど。)  

正規表現を使って日本語を消す。  
型式をでgroup化して最頻値を取り、型式番号をキーに正規化する(車両名、メーカー、排気量、タイプは別のファイルにしてしまう)

In [347]:
df.色.value_counts()

色
黒                 930
白                 431
青                 414
-                 369
赤                 349
                 ... 
緑/銀                 1
ｷﾔﾝﾃﾞｲﾄｰﾝﾌﾞﾗｳﾝ      1
メタリックブルー            1
白銀                  1
艶消し赤                1
Name: count, Length: 812, dtype: int64

In [349]:
name_list = df['車両名'].value_counts

数字、英字、記号、カタカナは半角に統一して、扱いやすくする。

In [354]:
#全角を半角にする関数
def zen_to_han(text):
    text = mojimoji.zen_to_han(text)
    return text
df['車両名'] = df['車両名'].apply(zen_to_han)
#車両名についてしまったメーカーを退けてあげる関数
def maker_replace(text):
    maker_list=list(df.メーカー.value_counts().index)
    for maker in maker_list:
        text = text.replace(maker,'')
    return text

df['車両名']=df['車両名'].apply(maker_replace)

漢字ひらがな・記号などはバイクの名前に関わらないため削除

In [355]:
#【】で囲まれた部分を外す
def marutoku_replace(text):
    text=re.sub('【+[ぁ-んァ-ヶｱ-ﾝﾞﾟ一-龠]+】','',text)
    text = re.sub('[\\-,.!?_…・×、。「」\\(\\)\\【\\】＃＠@]', '',text)
    pattern = re.compile(r'並行輸入[ ･]新車')
    text = re.sub(pattern,'',text)
    text=re.sub('[ぁ-ん一-龠]','',text)
    return text
df['車両名']=df['車両名'].apply(marutoku_replace)

海外使用モデルや新しいモデルには型式番号がないケースがある  
海外モデルには判別できる様に一般呼称を入れておく、  
--> 強引だがCRL190L、KTM1290は排気量が独特なので、メーカー,排気量,型式が-で当てはめる    
加えてCRL190はトリコロールしかない様なのでトリコロールで統一する。  
その他は一個づつしかないため、手打ちで埋める

In [356]:
df.query('排気量==190')

,id,価格,車両名,メーカー,型式番号,モデル年,排気量,色,タイプ,走行距離,保証期間,在庫店舗,店舗在庫
253,2100005316323,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,新横浜店,0120-37-8190
779,2100005316538,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,荒川沖店,0120-37-8190
951,2100005316316,440000,CRF190L,HONDA,-,2021,190,トリコロールカラー,オフロード/モタード,0,7,大阪東住吉店,0120-37-8190
1109,2100005316767,440000,ﾏﾙ･CRF190L14Lﾀﾝｸ145kg､ﾐﾆｱﾄﾞﾍﾞﾝﾁｬｰ,HONDA,-,2021,190,トリコロール,オフロード/モタード,1,7,高松店,0120-37-8190
1114,2100005316385,440000,CRF190L,HONDA,-,2021,190,-,オフロード/モタード,0,7,GLOBO蘇我店,0120-37-8190
1662,2100005316668,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,北九州店,0120-37-8190
1883,2100005315623,440000,CRF190L,HONDA,-,不明,190,トリコロール,オフロード/モタード,0,7,上尾店,0120-37-8190
1925,2100005316293,440000,CRF190LABS,HONDA,-,2021,190,TRICOLOR,オフロード/モタード,0,7,東松山店,0120-37-8190
2371,2100005316361,440000,CRF190L,HONDA,-,2021,190,トリコ,オフロード/モタード,1,7,名古屋守山店,0120-37-8190
2660,2100005315951,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,相模大野店,0120-37-8190


In [357]:
df[df['型式番号']=='-']

,id,価格,車両名,メーカー,型式番号,モデル年,排気量,色,タイプ,走行距離,保証期間,在庫店舗,店舗在庫
253,2100005316323,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,新横浜店,0120-37-8190
520,2100006264494,178000,EN1253F,SUZUKI,-,-,125,赤,ネイキッド,5909,3,久留米店,0120-37-8190
779,2100005316538,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,荒川沖店,0120-37-8190
951,2100005316316,440000,CRF190L,HONDA,-,2021,190,トリコロールカラー,オフロード/モタード,0,7,大阪東住吉店,0120-37-8190
1109,2100005316767,440000,ﾏﾙ･CRF190L14Lﾀﾝｸ145kg､ﾐﾆｱﾄﾞﾍﾞﾝﾁｬｰ,HONDA,-,2021,190,トリコロール,オフロード/モタード,1,7,高松店,0120-37-8190
1114,2100005316385,440000,CRF190L,HONDA,-,2021,190,-,オフロード/モタード,0,7,GLOBO蘇我店,0120-37-8190
1323,2100006052305,1198000,XL750ﾄﾗﾝｻﾞﾙﾌﾟ,HONDA,-,2023,750,白,スポーツ/ツアラー,470,7,寝屋川店,0120-37-8190
1662,2100005316668,440000,CRF190L,HONDA,-,2021,190,トリコロール,オフロード/モタード,0,7,北九州店,0120-37-8190
1774,2100005461924,1458000,ﾚﾌﾞﾙ1100T,HONDA,-,2023,1100,黒,アメリカン/クルーザー,414,7,GLOBO蘇我店,0120-37-8190
1883,2100005315623,440000,CRF190L,HONDA,-,不明,190,トリコロール,オフロード/モタード,0,7,上尾店,0120-37-8190


In [358]:
#CRL190Lの情報を入力
df.loc[(df['型式番号']=='-')&(df['排気量']==190),'型式番号']='CRF190L'
df.loc[(df['型式番号']=='CRF190L')&(df['排気量']==190),'色']='トリコロール'
#自動化する際はCRL190Lを名前に含み、方式-、排気量190だったらといった条件付けをすれば良いと考えている。

In [359]:
#KTM1290の情報を入力
df.loc[(df['型式番号']=='-')&(df['排気量']==1290),['車両名','型式番号']]='1290 SUPER DUKE GT'
df.loc[(df['型式番号']=='1290 SUPER DUKE GT')&(df['排気量']==1290),'色']='オレンジ'

In [373]:
#この７個は手打ちで修正する。
df[df['型式番号']=='-']

,id,価格,車両名,メーカー,型式番号,モデル年,排気量,色,タイプ,走行距離,保証期間,在庫店舗,店舗在庫
520,2100006264494,178000,EN1253F,SUZUKI,-,-,125,赤,ネイキッド,5909,3,久留米店,0120-37-8190
1323,2100006052305,1198000,XL750ﾄﾗﾝｻﾞﾙﾌﾟ,HONDA,-,2023,750,白,スポーツ/ツアラー,470,7,寝屋川店,0120-37-8190
1774,2100005461924,1458000,ﾚﾌﾞﾙ1100T,HONDA,-,2023,1100,黒,アメリカン/クルーザー,414,7,GLOBO蘇我店,0120-37-8190
2033,2100005355766,1378000,Bonneville T120,Triumph,-,2018,1200,ガンメタ,ネイキッド,3182,5,小山店,0120-37-8190
2673,2100003463807,2428000,R1250GS,BMW,-,2021,1250,ブラック／イエロー,オフロード/モタード,2533,7,平塚第2ショールーム,0120-37-8190
3107,2100005862400,1398000,ﾎﾞﾝﾈﾋﾞﾙT120classicalstylecool,Triumph,-,2020,650,黒,ネイキッド,1707,5,姫路店,0120-37-8190
4248,2100005587365,2900000,R1250GS,BMW,-,2021,1250,ブラック/イエロー,オフロード/モタード,67,7,荒川沖店,0120-37-8190


一度csvファイルにし、上記7つの型式番号と名前を入力

In [369]:
#df.to_csv('data_crf_ktm_reshape.csv')

csvファイルを再度読み込む

In [480]:

df_2 = pd.read_csv('data_crf_ktm_reshape.csv',index_col=0)

In [481]:
df_2

,id,価格,車両名,メーカー,型式番号,モデル年,排気量,色,タイプ,走行距離,保証期間,在庫店舗,店舗在庫
0,2100006155617,118000,ﾄｩﾃﾞｲ,HONDA,JBH-AF67,2012,50,黒,スクーター/ビッグスクーター,2742,3,京都店,0120-37-8190
1,2100004366077,1378000,CB400SFV4ABS,HONDA,EBL-NC42,2019,400,黒,ネイキッド,776,5,仙台店,0120-37-8190
2,2100005457217,348000,Z250,KAWASAKI,JBK-ER250C,2013,250,白,ネイキッド,11979,1,りんくうシークル店,0120-37-8190
3,2100005410007,578000,YZFR3 Monster Energy Yamaha MotoGP Edition,YAMAHA,EBL-RH07J,2020,320,黒,スポーツ/ツアラー,4500,5,金沢店,0120-37-8190
4,2100005346689,568000,SVARTPILEN 250 2021ﾓﾃﾞﾙ/ﾊﾞｰｴﾝﾄﾞﾐﾗｰ,Husqvarna,JUE4,2021,250,黒,ネイキッド,5688,5,Husqvarna Motorcycles いわき店,0120-37-8190
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6102,2100006097726,1018000,YZFR7,YAMAHA,8BL-RM39J,2022,700,青/黒,スポーツ/ツアラー,2964,7,広島店,0120-37-8190
6103,2100006214697,798000,DRZ400SM★★,SUZUKI,BC-SK44A,2008,400,黒,オフロード/モタード,11745,3,ラパークいわき店,0120-37-8190
6104,2100006202625,1198000,CBR1000RR ABS Special Edition,HONDA,EBL-SC59,2013,1000,バイブラントオレンジ,スポーツ/ツアラー,23013,1,久留米店,0120-37-8190
6105,2100006271508,466000,YZFR252019ﾓﾃﾞﾙ/ﾏﾙﾁﾊﾞｰ/USB,YAMAHA,JBK-RG10J,2019,250,艶消し赤,スポーツ/ツアラー,5895,5,東大阪店,0120-37-8190


In [482]:
df_2[df_2['型式番号']=='2BJ-JA45'].車両名.mode()

0     ｸﾛｽｶﾌﾞ110
Name: 車両名, dtype: object

In [483]:
df_2[df_2['型式番号']=='-']

,id,価格,車両名,メーカー,型式番号,モデル年,排気量,色,タイプ,走行距離,保証期間,在庫店舗,店舗在庫


型式に対して車両呼称は一意に決まることを利用して、  
型式番号でgroup化した車両名の最頻値を今回は車両名として使うことを考えた。  
データをいじっていてわかったが、タイプもたまに違う時があるため、これも正規化し最頻値を使う。


In [484]:
df_2['型式番号']=df_2['型式番号'].apply(zen_to_han)

In [485]:
df_2_g_test=df_2.groupby('型式番号')[['メーカー','車両名','タイプ','排気量']].apply(lambda x :x.mode()).reset_index()

In [486]:
df_2_g_test=df_2_g_test[df_2_g_test['level_1']==0]

In [404]:
#df_2_g_test.to_csv('test.csv')

In [487]:
df_2_g_test['車両名']

0                                    M1000RR
1                                    R1250GS
2                                  F900R ABS
3                                R18 Classic
4                         1290 SUPER DUKE GT
                        ...                 
1157                                   Ninja
1160             Ninja ZX10R ABS KRT Edition
1161     ZZR1400 2007ﾓﾃﾞﾙｶﾜｻｷZZRｼﾘｰｽﾞﾒｶﾞﾂｱﾗｰ
1162                         Ninja ZX14R ABS
1163             Ninja ZX14R Special Edition
Name: 車両名, Length: 854, dtype: object

In [488]:
df_2_g_test['車両名']=df_2_g_test['車両名'].str.replace('11ﾌﾟﾗｲｽ','')


In [489]:
#20XX ~ を除ける正規表現
def replace_subdata(text):
    text = re.sub(r'20\d{2}.*', '', text)
    text = re.sub('[\\-,.!?_…・×、。★「」\\(\\)\\【\\】[\\]＃＠@]', '',text)
    text = re.sub(r'ｴﾝｼﾞﾝ.*','',text)
    text = re.sub(r'ｸﾞﾘｯﾌﾟ.*','',text)
    text = text.split('■')[0]
    text = text.split('/')[0]
    
    return text
df_2_g_test['車両名'] = df_2_g_test['車両名'].apply(replace_subdata)

In [490]:
df_2_g_test=df_2_g_test.reset_index()
df_2_g_test.drop(columns=['level_1','index'],inplace=True)
df_2_g_test.to_csv('test_2.csv')

In [491]:
df_2_g_test

,型式番号,メーカー,車両名,タイプ,排気量
0,0E71,BMW,M1000RR,スポーツ/ツアラー,1000.0
1,0J51,BMW,R1250GS,オフロード/モタード,1250.0
2,0K11,BMW,F900R ABS,ネイキッド,900.0
3,0L11,BMW,R18 Classic,アメリカン/クルーザー,1800.0
4,1290 SUPER DUKE GT,KTM,1290 SUPER DUKE GT,ネイキッド,1290.0
...,...,...,...,...,...
849,ZXT00N,KAWASAKI,Ninja,スポーツ/ツアラー,1000.0
850,ZXT00S,KAWASAKI,Ninja ZX10R ABS KRT Edition,スポーツ/ツアラー,1000.0
851,ZXT40A,KAWASAKI,ZZR1400,スポーツ/ツアラー,1400.0
852,ZXT40E,KAWASAKI,Ninja ZX14R ABS,スポーツ/ツアラー,1400.0


In [3]:
#一部気になったところを修正
data = pd.read_csv('../data/整形後データ/modelnumber.csv',index_col=0)

In [4]:
data

,型式番号,メーカー,車両名,タイプ,排気量
0,0E71,BMW,M1000RR,スポーツ/ツアラー,1000.0
1,0J51,BMW,R1250GS,オフロード/モタード,1250.0
2,0K11,BMW,F900R ABS,ネイキッド,900.0
3,0L11,BMW,R18 Classic,アメリカン/クルーザー,1800.0
4,1290 SUPER DUKE GT,KTM,1290 SUPER DUKE GT,ネイキッド,1290.0
...,...,...,...,...,...
849,ZXT00N,KAWASAKI,Ninja,スポーツ/ツアラー,1000.0
850,ZXT00S,KAWASAKI,Ninja ZX10R ABS KRT Edition,スポーツ/ツアラー,1000.0
851,ZXT40A,KAWASAKI,ZZR1400,スポーツ/ツアラー,1400.0
852,ZXT40E,KAWASAKI,Ninja ZX14R ABS,スポーツ/ツアラー,1400.0


In [6]:
data['車両名']=data['車両名'].str.replace('☆','')

In [11]:
data.to_csv('test.csv')

In [13]:
def replace_subdata(text):
    text = re.sub(r'20\d{2}.*', '', text)
    text = re.sub('[\\-,.!?_…・×、。★「」\\(\\)\\【\\】[\\]＃＠@]', '',text)
    text = re.sub(r'(ｴﾝｼﾞﾝ|ｸﾞﾘｯﾌﾟ|ﾏﾌﾗｰ|ﾐﾗｰ|ﾜﾝｵｰﾅｰ|ﾀﾝﾃﾞﾑﾊﾞｰ).*', '', text)
    text = text.split('■')[0]
    text = text.split('/')[0]
    return text

data['車両名']=data['車両名'].apply(replace_subdata)

In [14]:
data.to_csv('../data/整形後データ/modelnumber.csv')

# EDA時に発生した問題を修正  
EDAを行う際に重要変数の確認の際にlightgbmをかけた時に下記の問題が発生したため訂正した。  
* 走行距離に減算車、距離疑義車、-などのデータあり-->データとして使えないので欠損値にした。
* 走行距離にメーター交換車 交換前xxkm 交換後yykmというデータあり-->交換前後の和を取り特徴量とした。  
* モデル年の中に空白があるデータあり(例:'2021 年')-->空白を削除した。  
* モデル年、色に'-'のデータあり-->精度の悪化につながるため、削除した
* 保証期間'-'のデータあり-->保証期間0と考え、0に置き換えた

In [55]:
data_info = pd.read_csv('../data/整形後データ/bike_info.csv',index_col=0)

In [56]:
#減算車は距離が使えないので欠損値にする
data_info['走行距離']=data_info['走行距離'].replace('減算車',np.nan)
data_info['走行距離']=data_info['走行距離'].replace('距離疑義車',np.nan)

In [61]:
#モデル年の中に空白があったので削除
data_info['モデル年'] = data_info['モデル年'].str.replace(r'\s+', '', regex=True)
#モデル年、色も'-'はNanにする
data_info['モデル年']=data_info['モデル年'].replace('-',np.nan)
data_info['モデル年']=data_info['モデル年'].replace('不明',np.nan)
data_info['色']=data_info['色'].replace('-',np.nan)

In [58]:
#保証期間が'-'のものは0を入れておく
data_info['保証期間']=data_info['保証期間'].replace('-',0)

In [63]:
data_info.isnull().sum()

id        0
価格        0
型式番号      0
モデル年     35
色       367
走行距離    104
保証期間      0
店舗id      0
dtype: int64

In [62]:
#data_info.to_csv('../data/整形後データ/bike_info.csv')

In [116]:
data_info['走行距離'] = data_info['走行距離'].fillna('-')

In [117]:
def replace_dis(text):
    match = re.search(r'メーター交換歴車（交換前(\d+) 交換後(\d+)）',text)
    if match is None :
        text = text
    elif match :
        text = int(match.group(1)) + int(match.group(2))
    return text


data_info['走行距離'] = data_info['走行距離'].apply(replace_dis)

In [120]:
data_info['走行距離']=data_info['走行距離'].replace('-',np.nan)

In [122]:
data_info.to_csv('../data/整形後データ/bike_info.csv')